## Leaky integrated firing neuron model  
## Few feautures :
### Spatial network, no memory term but exponential decay of cumulative dose

This is a minor change of the Dodds and Watts model  
Implement adaptive rewiring and generate statistics

In [1]:
#importing the required libraries

import networkx as nx
import matplotlib.pyplot as plt #for plotting
import numpy as np #for using arrays and vectorising the code wherever possible
import scipy
import random
# from numba import jit #numba precomplier to make the code faster
import pandas as pd
import copy
import warnings
warnings.filterwarnings('ignore')

# Variables used in the simulation

In [2]:
N = 100
gamma = 1 #decay of cumulative dose per unit time (second)
gamma = gamma
T = 100 #total number of seconds
dt = 0.05 #seconds
'''number of simulation steps is T/dt'''

'''Dodds watts parameters'''
p = 1 #rate of dose transfer (per unit time (second))
p = p*dt #rate of dose transfer per unit simulation step
r = 1/dt
r = r*dt
rho = 1/dt
rho = rho*dt
# r = 0.5
# r = r*dt

d_star = 1
# dose_quantity = 1
D_ini = 3

'''network parameters'''
circ_rad = 100 #for a network in which each node is connected to other nodes withing a circle of radius circ_rad
nn = 10 #the number of nearest neighbours(i.e. k) in knn
average_degree = 6
total_edges = N*2

'''rewiring parameters'''
rew_r = 18
density = 1/N
# p_rew = 5 #rate of rewiring(per unit time(second))
# p_rew = p_rew*dt

# Creating the network. 
### The network outputs coordinates, distance matrix, edge_list and adjacency matrix

In [3]:
'''
obtains the number of nodes N
generates N points
calculates euclidean distance between each pair of points
returns the coordinates of the points and the distance matrix which is N*N
'''
def calc_dist(N) :
    coords = []
    for counter in range(N) :
        coords.append((N*np.random.random(),N*np.random.random()))

    dist_mat = scipy.spatial.distance.cdist(coords,coords)
    return dist_mat,coords

### 1. metric network

In [4]:
'''network = connect to points withing a circle of radius'''
#creates a link between points/nodes which satisfies the conditions of the network
def coupling(dist_mat,circ_rad) :
    edge_list = []
    edge_mask = (dist_mat<circ_rad) & (dist_mat>0)
    edge_list.extend(np.ndarray.tolist(np.transpose(np.where(edge_mask))))
#     print(type(np.ndarray.tolist(np.transpose(np.where(edge_mask)))))
    return edge_list

### 2. K nearest neighbour network

In [5]:
'''network = knn'''
def knn(dist_mat,nn,N) :
    near_neigh = np.argsort(dist_mat)
    selec_near_neigh = np.zeros((N,nn))
    selec_near_neigh = near_neigh[:,0:nn+1]

    edge_list = []
    for i in range(N) :
        for j in range(1,nn+1) :
            link = [i,selec_near_neigh[i,j]]
            edge_list.append(link)

    return edge_list

### 3. Random Network : GNP type

In [6]:
'''random network'''
def rand_network(average_degree,N) :
    z1 = np.random.uniform(size = (N,N))
    E,F = np.meshgrid(np.arange(0,N),np.arange(0,N))
    mask = ((average_degree/N) > z1) & (E!=F)
    adjacency_matrix = np.int64(np.zeros(shape=(N,N)))
    adjacency_matrix[mask] = np.int64(1)
    edge_list = []
    edge_list.extend(np.ndarray.tolist(np.transpose(np.where(adjacency_matrix==1))))
    return edge_list,adjacency_matrix

### 4. Random Network : GNM type

In [7]:
def rand_net_gnm(total_edges,N) :
    the_graph = nx.gnm_random_graph(N, total_edges,directed=True)
    adjacency_matrix = nx.adjacency_matrix(the_graph)
    adjacency_matrix = np.asarray(adjacency_matrix.todense())
    np.fill_diagonal(adjacency_matrix,0)
    edge_list = []
    edge_list.extend(np.ndarray.tolist(np.transpose(np.where(adjacency_matrix==1))))
    return edge_list

# Function defintions required for the main part of the script

In [8]:
# # @jit(nopython=True)
# '''infecting the left part of space.'''
# def left_part_infec(N) :
#     x_coord = []
#     y_coord = []
#     for j in range(len(coords)) :
#         x_coord.append(coords[j][0])
#         y_coord.append(coords[j][1])
#     x_coord = np.asarray(x_coord)
#     y_coord = np.asarray(y_coord)
#     points = np.asarray(np.where(x_coord<(N/5))) 
#     indi_state[points.T] = 2

#     return indi_state

In [9]:
'''infecting connected nodes'''
def InfectNetworkNeighbors(net,seed_node,init_infected_nodes):
     # if in bulk find one node randomly, and infect its neighbours
    infected_nodes = set()
    candidate_nodes = set()
    explored_nodes = set()

    #pick the seed node
    infected_nodes.add(seed_node)
    explored_nodes.add(seed_node)

    curr_node=seed_node

    #add its neighbors to the list of candidates
    for n in net.neighbors(curr_node):
        candidate_nodes.add(int(n))
    #print( curr_node)
    #print( candidate_nodes)

    #while we need to select more nodes...
    while len(infected_nodes) < init_infected_nodes:

        #if there are candidate nodes, select one of them
        if(len(candidate_nodes) > 0):
            new_node = np.random.choice(list(candidate_nodes),1)[0]
            infected_nodes.add(new_node)
            candidate_nodes.remove(new_node)

        elif len(infected_nodes - explored_nodes) > 0:
            curr_node = np.random.choice(list(infected_nodes -
            explored_nodes),1)[0]
            explored_nodes.add(curr_node)
            for n in set(net.neighbors(curr_node)) - infected_nodes:
                candidate_nodes.add(n)

        else:
#             print('Initial node infection step failed')
            return None
    return infected_nodes

In [10]:
''' function to transfer doses'''
# @jit(nopython=True)
def dose(adj_mat,p,partner_state_prev,d) :

    z1 = np.random.uniform(size=(len(adj_mat),len(adj_mat[0])))
    
    dose_transfer_mask = p > z1
    mod_adj_mat = np.multiply(adj_mat,z1) #modifying adjacency matrix to include the proability of dose transfer
    dose_mask = (p>mod_adj_mat) & (adj_mat!=0) & (partner_state_prev == 2)
    d[dose_mask] = dose_quantity #whenever it is proabable, dose transfer occurs
    
    return d

In [11]:
def rew(p_rew,adjacency_matrix,indi_state_for_rew_prev,OD_mesh2,lamb_da,density,rew_r) :
    
    z1_rew_prew = np.random.uniform(size=(len(adj_mat),len(adj_mat[0])))
    z1_rew_lambda = np.random.uniform(size=(len(adj_mat),len(adj_mat[0])))
    
    rew_mask_plus = (p_rew > z1_rew_prew)  & (indi_state_for_rew_prev == 1) & (dist_mat <= rew_r)
    #create links. Dont break any.
    adjacency_matrix[rew_mask_plus] = 1
    rew_mask_minus1 = (p_rew > z1_rew_prew) & (indi_state_for_rew_prev == 2)
    #break links. Dont create any.
    adjacency_matrix[rew_mask_minus1] = 0
    rew_mask_minus2 = ((lamb_da*OD_mesh2) > z1_rew_lambda) & (indi_state_for_rew_prev == 1)
    adjacency_matrix[rew_mask_minus2] = 0
    
    np.fill_diagonal(adjacency_matrix,0)
    
    return adjacency_matrix

In [12]:
'''function to update cumulative doses'''
def cumu_dose(d,D_prev,gamma) :
    I = d.sum(axis=1).reshape(N,1)
#     I = 0
    D = (D_prev - (gamma*D_prev*dt)) + I
    return D

In [13]:
'''function to update the states of the nodes of the system'''
def upd_indi_state(D,d_star,indi_state_prev) :
    
    z2 = np.random.uniform(size=(N,1))
    z3 = np.random.uniform(size=(N,1))
    
    indi_state = indi_state_prev
    
    indi_state_mask1 = (D>=d_star) & (indi_state_prev==1)
    indi_state[indi_state_mask1] = 2
    
    indi_state_mask21 = (D<d_star) & (indi_state_prev==2) & (r>=z2) & (rho>=z3)
    indi_state[indi_state_mask21] = 1
    
    indi_state_mask22 = (D<d_star) & (indi_state_prev==2) & (r>=z2) & (rho<z3)
    indi_state[indi_state_mask22] = 3
    
    indi_state_mask23 = (D<d_star) & (indi_state_prev==2) & (r<z2)
    indi_state[indi_state_mask23] = 2
    
    return indi_state

In [14]:
'''function to get/identify the infected nodes and the suceptible nodes'''
def states(indi_state) :
    infec_indi = []
    suscep_indi = []
    infec_indi = np.transpose(np.where(indi_state==2))
    suscep_indi = np.transpose(np.where(indi_state==1))
    return infec_indi,suscep_indi

In [15]:
def networkx_graph(coords,edge_list) :
    G = nx.DiGraph()
    pos = {(i): (coords[i][0],coords[i][1]) for i in range(N)}
    G.add_nodes_from(pos.keys())
    G.add_edges_from(edge_list)
    return G

In [16]:
def edge_from_adj_mat(adj_mat_list,q):
    edge_list = []
    edge_list.extend(np.ndarray.tolist(np.transpose(np.where(adj_mat_list[q]==1))))
    return edge_list

# The main part of the script

In [17]:
#infected state time series data frame
#columns indicate the time steps
timeseries_infec_frac = pd.DataFrame()

#in degree as columns and rows as nodes
timeseries_in_degree = pd.DataFrame() 

#out degree as columns and rows as nodes
timeseries_out_degree = pd.DataFrame()

#strongly connected components as columns
timeseries_connec_comps = pd.DataFrame()

In [18]:
vals = round(1/((density)*(np.pi)*(rew_r*rew_r)),3)
p_rew_vals = [round(vals,3),round(10*vals,3),round(100*vals,3),round(1000*vals,3)]
start_vals = [1,10]
dose_quantity_vals = [0.5,2]

In [19]:
for sim in range(70,70+100) :
    '''returns coordinates, distance matrix, edge_list and the adjacency matrix'''
    dist_mat,coords = calc_dist(N) #node placement
    edge_list = knn(dist_mat,nn,N)
    org_adj_mat = np.int64(np.zeros((N,N)))
    for i in range(len(edge_list)):
        org_adj_mat[edge_list[i][0],edge_list[i][1]] = np.int64(1)
    df_row = 0
    for p_rew in p_rew_vals : #rate of rewiring (per unit time (second))
        lamb_da = p_rew/10
        lamb_da = lamb_da*dt
        p_rew = p_rew*dt
        for start in start_vals : #rate of dose transfer (per unit time (second))
            for dose_quantity in dose_quantity_vals : #number of nodes to infect initially
                print('(sim,lamb_da,p_rew,start,dose_quantity) = ', (sim,lamb_da/dt,p_rew/dt,start,dose_quantity))

                '''clearing exsiting data frames and creating new ones '''
                timeseries_infec_frac = pd.DataFrame()
                timeseries_in_degree = pd.DataFrame()
                timeseries_out_degree = pd.DataFrame()
                timeseries_connec_comps = pd.DataFrame()

                q = 0 #to generate new edge_list from the new adjacency matrix 
                t = np.arange(0,T,dt)

    #             numpy_array = np.zeros((N,len(t)),dtype = np.int64) #stores the states of the nodes. Rows are nodes and columns are time steps
                D_array = np.zeros((N,len(t))) #stores the cumulative doses. Rows are nodes and columns are time steps

                adj_mat_list = []
                adj_mat = copy.deepcopy(org_adj_mat)
                adj_mat_new = copy.deepcopy(adj_mat)
                adj_mat_list.append(adj_mat_new) #list of arrays which shows the time series of the adjacency matrix

                #coords remain the same. Get new edge_list from the latest adjacency matrix
                edge_list = edge_from_adj_mat(adj_mat_list,q)
#                 print(edge_list)
                G = networkx_graph(coords,edge_list) #networkx graph

                '''from G, obtain the in degree and the out degree'''
                timeseries_in_degree.loc[:,0] = np.asarray([val for (node, val) in G.in_degree()])
                timeseries_out_degree.loc[:,0] = np.asarray([val for (node, val) in G.out_degree()])

                '''strongly and weekely connected components'''
                timeseries_connec_comps.loc[0,0] = nx.number_connected_components(G.to_undirected())

                '''choosing initially infected nodes'''
                indi_state = np.random.randint(1,2,size=(N,1))
                infected_nodes = None
                while infected_nodes is None : #infecting 'start' number of network neighbours
                    infected_nodes = InfectNetworkNeighbors(G,np.random.randint(N),start)      
                print(infected_nodes)
                infected_nodes = np.asarray(list(infected_nodes)).reshape(len(infected_nodes),1)
                indi_state[infected_nodes[:,0],0] = 2

                A,B = np.meshgrid(indi_state,indi_state) 
                partner_state = copy.deepcopy(A)
                indi_state_for_rew = copy.deepcopy(B)

    #             numpy_array [:,0] = indi_state[:,0]

                indi_state_prev = copy.deepcopy(indi_state)
                partner_state_prev = copy.deepcopy(partner_state)
                indi_state_for_rew_prev = copy.deepcopy(indi_state_for_rew)

                d = np.zeros((len(adj_mat),len(adj_mat[0])))

                D = np.zeros((N,1))
                D[np.where(indi_state==2)] = D_ini
                D_array[:,0] = D[:,0]
                D_prev = copy.deepcopy(D)

                infec_frac = np.count_nonzero(indi_state == 2)/N
                timeseries_infec_frac.loc[df_row,0] = infec_frac
                '''the main part of the simulation'''
                counter = 0
                df_col = 1
                for t in np.arange(dt,T,dt) :
                    q = q + 1 
                    counter = counter + 1
                    infec_indi = []
                    suscep_indi = []
                    d = np.zeros((len(adj_mat),len(adj_mat[0])))
                    d = dose(adj_mat,p,partner_state_prev,d)

                    OD_array = np.asarray([val for (node, val) in G.out_degree()])
                    OD_mesh1,OD_mesh2 = np.meshgrid(OD_array,OD_array)
                    adj_mat = rew(p_rew,adj_mat,indi_state_for_rew_prev,OD_mesh2,lamb_da,density,rew_r)
                    adj_mat_new = copy.deepcopy(adj_mat)
                    adj_mat_list.append(adj_mat_new)

                    #coords remain the same. Get new edge_list from the latest adjacency matrix
                    edge_list = edge_from_adj_mat(adj_mat_list,q)
                    G = networkx_graph(coords,edge_list) #networkx graph

                    '''from G, obtain the in degree and the out degree'''
                    timeseries_in_degree.loc[:,df_col] = np.asarray([val for (node, val) in G.in_degree()])
                    timeseries_out_degree.loc[:,df_col] = np.asarray([val for (node, val) in G.out_degree()])

                    '''strongly and weekely connected components'''
                    timeseries_connec_comps.loc[df_row,df_col] = nx.number_connected_components(G.to_undirected())

                    D = cumu_dose(d,D_prev,gamma)
                    D_array[:,counter] = D[:,0]

                    indi_state = upd_indi_state(D,d_star,indi_state_prev)
                    infec_frac = np.count_nonzero(indi_state == 2)/N
                    timeseries_infec_frac.loc[df_row,df_col] = infec_frac

    #                 numpy_array[:,counter] = indi_state[:,0]
                    A,B = np.meshgrid(indi_state,indi_state)

                    infec_indi, suscep_indi = states(indi_state)

                    partner_state = copy.deepcopy(A)
                    indi_state_for_rew = copy.deepcopy(B)
                    indi_state_prev = copy.deepcopy(indi_state)
                    partner_state_prev = copy.deepcopy(partner_state)
                    indi_state_for_rew_prev = copy.deepcopy(indi_state_for_rew)

                    D_prev = copy.deepcopy(D)
                    D = np.zeros((N,1))
                    df_col = df_col + 1 

    #             df_row = df_row + 1 #going to the next row of the df to store timeseries of next (p,start)

                filename_infec_frac = 'infec_frac.h5'
                filename_in_deg = 'in_deg.h5'
                filename_out_deg = 'out_deg.h5'
                filename_connec_comps = 'connec_comps.h5'
                p_rew_val = '%g'%(p_rew/dt)
                key_val = 'sim_'+str(sim)+'_p_rew_'+str(p_rew_val)+'_start_'+str(start)+'_dq_'+str(dose_quantity)
#                 print(key_val)
                timeseries_infec_frac.to_hdf(filename_infec_frac, key = key_val, mode='a')
                timeseries_in_degree.to_hdf(filename_in_deg, key = key_val, mode='a')
                timeseries_out_degree.to_hdf(filename_out_deg, key = key_val, mode='a')
                timeseries_connec_comps.to_hdf(filename_connec_comps, key = key_val, mode='a')
                df_row = 0

(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.0098, 0.098, 1, 0.5)
{5}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.0098, 0.098, 1, 2)
{84}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.0098, 0.098, 10, 0.5)
{0, 64, 37, 40, 45, 46, 60, 81, 21, 92}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.0098, 0.098, 10, 2)
{1, 66, 44, 77, 48, 80, 56, 24, 88, 31}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.098, 0.98, 1, 0.5)
{31}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.098, 0.98, 1, 2)
{94}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.098, 0.98, 10, 0.5)
{34, 69, 41, 43, 12, 20, 55, 27, 93, 62}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.098, 0.98, 10, 2)
{1, 66, 44, 77, 80, 48, 56, 24, 88, 31}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.9800000000000001, 9.8, 1, 0.5)
{35}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.9800000000000001, 9.8, 1, 2)
{76}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.9800000000000001, 9.8, 10, 0.5)
{0, 34, 6, 7

(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 0.9800000000000001, 9.8, 1, 0.5)
{39}
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 0.9800000000000001, 9.8, 1, 2)
{54}
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 0.9800000000000001, 9.8, 10, 0.5)
{32, 67, 52, 54, 88, 89, 58, 63, 30, 31}
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 0.9800000000000001, 9.8, 10, 2)
{68, 5, 74, 43, 42, 77, 45, 10, 27, 62}
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 9.8, 98.0, 1, 0.5)
{88}
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 9.8, 98.0, 1, 2)
{79}
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 9.8, 98.0, 10, 0.5)
{64, 98, 35, 6, 7, 8, 16, 92, 94, 95}
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 9.8, 98.0, 10, 2)
{65, 68, 7, 8, 16, 48, 83, 53, 92, 94}
(sim,lamb_da,p_rew,start,dose_quantity) =  (76, 0.0098, 0.098, 1, 0.5)
{92}
(sim,lamb_da,p_rew,start,dose_quantity) =  (76, 0.0098, 0.098, 1, 2)
{79}
(sim,lamb_da,p_rew,start,dose_quantity) =  (76, 0.0098, 0.098, 10, 0.5)
{98, 2, 6, 7

(sim,lamb_da,p_rew,start,dose_quantity) =  (81, 0.0098, 0.098, 1, 0.5)
{12}
(sim,lamb_da,p_rew,start,dose_quantity) =  (81, 0.0098, 0.098, 1, 2)
{69}
(sim,lamb_da,p_rew,start,dose_quantity) =  (81, 0.0098, 0.098, 10, 0.5)
{41, 12, 14, 78, 48, 19, 51, 24, 26, 62}
(sim,lamb_da,p_rew,start,dose_quantity) =  (81, 0.0098, 0.098, 10, 2)
{97, 68, 37, 38, 79, 17, 49, 25, 91, 92}
(sim,lamb_da,p_rew,start,dose_quantity) =  (81, 0.098, 0.98, 1, 0.5)
{9}
(sim,lamb_da,p_rew,start,dose_quantity) =  (81, 0.098, 0.98, 1, 2)
{17}
(sim,lamb_da,p_rew,start,dose_quantity) =  (81, 0.098, 0.98, 10, 0.5)
{32, 39, 42, 76, 16, 85, 54, 88, 60, 30}
(sim,lamb_da,p_rew,start,dose_quantity) =  (81, 0.098, 0.98, 10, 2)
{97, 99, 35, 5, 13, 81, 22, 25, 27, 94}
(sim,lamb_da,p_rew,start,dose_quantity) =  (81, 0.9800000000000001, 9.8, 1, 0.5)
{91}
(sim,lamb_da,p_rew,start,dose_quantity) =  (81, 0.9800000000000001, 9.8, 1, 2)
{57}
(sim,lamb_da,p_rew,start,dose_quantity) =  (81, 0.9800000000000001, 9.8, 10, 0.5)
{97, 68, 3

(sim,lamb_da,p_rew,start,dose_quantity) =  (86, 0.9800000000000001, 9.8, 1, 0.5)
{73}
(sim,lamb_da,p_rew,start,dose_quantity) =  (86, 0.9800000000000001, 9.8, 1, 2)
{24}
(sim,lamb_da,p_rew,start,dose_quantity) =  (86, 0.9800000000000001, 9.8, 10, 0.5)
{0, 42, 45, 46, 81, 18, 82, 17, 87, 90}
(sim,lamb_da,p_rew,start,dose_quantity) =  (86, 0.9800000000000001, 9.8, 10, 2)
{2, 35, 72, 8, 13, 60, 19, 20, 24, 28}
(sim,lamb_da,p_rew,start,dose_quantity) =  (86, 9.8, 98.0, 1, 0.5)
{3}
(sim,lamb_da,p_rew,start,dose_quantity) =  (86, 9.8, 98.0, 1, 2)
{34}
(sim,lamb_da,p_rew,start,dose_quantity) =  (86, 9.8, 98.0, 10, 0.5)
{0, 42, 76, 45, 46, 81, 87, 23, 90, 95}
(sim,lamb_da,p_rew,start,dose_quantity) =  (86, 9.8, 98.0, 10, 2)
{36, 42, 78, 50, 18, 86, 23, 56, 59, 95}
(sim,lamb_da,p_rew,start,dose_quantity) =  (87, 0.0098, 0.098, 1, 0.5)
{47}
(sim,lamb_da,p_rew,start,dose_quantity) =  (87, 0.0098, 0.098, 1, 2)
{65}
(sim,lamb_da,p_rew,start,dose_quantity) =  (87, 0.0098, 0.098, 10, 0.5)
{32, 37, 38

(sim,lamb_da,p_rew,start,dose_quantity) =  (92, 0.0098, 0.098, 1, 0.5)
{48}
(sim,lamb_da,p_rew,start,dose_quantity) =  (92, 0.0098, 0.098, 1, 2)
{48}
(sim,lamb_da,p_rew,start,dose_quantity) =  (92, 0.0098, 0.098, 10, 0.5)
{2, 6, 8, 73, 42, 12, 48, 82, 27, 93}
(sim,lamb_da,p_rew,start,dose_quantity) =  (92, 0.0098, 0.098, 10, 2)
{35, 68, 5, 72, 43, 76, 46, 51, 22, 28}
(sim,lamb_da,p_rew,start,dose_quantity) =  (92, 0.098, 0.98, 1, 0.5)
{72}
(sim,lamb_da,p_rew,start,dose_quantity) =  (92, 0.098, 0.98, 1, 2)
{4}
(sim,lamb_da,p_rew,start,dose_quantity) =  (92, 0.098, 0.98, 10, 0.5)
{2, 37, 6, 8, 73, 15, 18, 82, 26, 27}
(sim,lamb_da,p_rew,start,dose_quantity) =  (92, 0.098, 0.98, 10, 2)
{0, 39, 9, 75, 14, 16, 20, 56, 26, 91}
(sim,lamb_da,p_rew,start,dose_quantity) =  (92, 0.9800000000000001, 9.8, 1, 0.5)
{91}
(sim,lamb_da,p_rew,start,dose_quantity) =  (92, 0.9800000000000001, 9.8, 1, 2)
{89}
(sim,lamb_da,p_rew,start,dose_quantity) =  (92, 0.9800000000000001, 9.8, 10, 0.5)
{96, 2, 39, 42, 12

...

(sim,lamb_da,p_rew,start,dose_quantity) =  (146, 0.098, 0.98, 1, 0.5)
{92}
(sim,lamb_da,p_rew,start,dose_quantity) =  (146, 0.098, 0.98, 1, 2)
{38}
(sim,lamb_da,p_rew,start,dose_quantity) =  (146, 0.098, 0.98, 10, 0.5)
{2, 70, 41, 9, 43, 77, 15, 83, 22, 31}
(sim,lamb_da,p_rew,start,dose_quantity) =  (146, 0.098, 0.98, 10, 2)
{40, 27, 16, 17, 81, 86, 24, 89, 59, 93}
(sim,lamb_da,p_rew,start,dose_quantity) =  (146, 0.9800000000000001, 9.8, 1, 0.5)
{58}
(sim,lamb_da,p_rew,start,dose_quantity) =  (146, 0.9800000000000001, 9.8, 1, 2)
{66}
(sim,lamb_da,p_rew,start,dose_quantity) =  (146, 0.9800000000000001, 9.8, 10, 0.5)
{32, 10, 80, 82, 84, 85, 87, 55, 90, 29}
(sim,lamb_da,p_rew,start,dose_quantity) =  (146, 0.9800000000000001, 9.8, 10, 2)
{96, 67, 99, 36, 75, 14, 48, 19, 56, 60}
(sim,lamb_da,p_rew,start,dose_quantity) =  (146, 9.8, 98.0, 1, 0.5)
{67}
(sim,lamb_da,p_rew,start,dose_quantity) =  (146, 9.8, 98.0, 1, 2)
{46}
(sim,lamb_da,p_rew,start,dose_quantity) =  (146, 9.8, 98.0, 10, 0.5)
{

(sim,lamb_da,p_rew,start,dose_quantity) =  (151, 0.9800000000000001, 9.8, 10, 2)
{4, 70, 38, 39, 11, 44, 79, 49, 88, 92}
(sim,lamb_da,p_rew,start,dose_quantity) =  (151, 9.8, 98.0, 1, 0.5)
{14}
(sim,lamb_da,p_rew,start,dose_quantity) =  (151, 9.8, 98.0, 1, 2)
{66}
(sim,lamb_da,p_rew,start,dose_quantity) =  (151, 9.8, 98.0, 10, 0.5)
{1, 2, 35, 58, 10, 91, 82, 26, 59, 29}
(sim,lamb_da,p_rew,start,dose_quantity) =  (151, 9.8, 98.0, 10, 2)
{3, 4, 11, 14, 15, 49, 88, 92, 94, 95}
(sim,lamb_da,p_rew,start,dose_quantity) =  (152, 0.0098, 0.098, 1, 0.5)
{98}
(sim,lamb_da,p_rew,start,dose_quantity) =  (152, 0.0098, 0.098, 1, 2)
{54}
(sim,lamb_da,p_rew,start,dose_quantity) =  (152, 0.0098, 0.098, 10, 0.5)
{0, 65, 39, 79, 51, 53, 25, 91, 28, 30}
(sim,lamb_da,p_rew,start,dose_quantity) =  (152, 0.0098, 0.098, 10, 2)
{98, 35, 39, 74, 12, 79, 80, 17, 50, 29}
(sim,lamb_da,p_rew,start,dose_quantity) =  (152, 0.098, 0.98, 1, 0.5)
{46}
(sim,lamb_da,p_rew,start,dose_quantity) =  (152, 0.098, 0.98, 1, 2)
{

(sim,lamb_da,p_rew,start,dose_quantity) =  (157, 0.0098, 0.098, 10, 0.5)
{0, 1, 66, 78, 15, 82, 18, 61, 88, 93}
(sim,lamb_da,p_rew,start,dose_quantity) =  (157, 0.0098, 0.098, 10, 2)
{96, 99, 68, 69, 6, 7, 41, 26, 92, 29}
(sim,lamb_da,p_rew,start,dose_quantity) =  (157, 0.098, 0.98, 1, 0.5)
{66}
(sim,lamb_da,p_rew,start,dose_quantity) =  (157, 0.098, 0.98, 1, 2)
{85}
(sim,lamb_da,p_rew,start,dose_quantity) =  (157, 0.098, 0.98, 10, 0.5)
{36, 40, 73, 75, 49, 52, 21, 54, 86, 31}
(sim,lamb_da,p_rew,start,dose_quantity) =  (157, 0.098, 0.98, 10, 2)
{0, 66, 71, 15, 18, 93, 88, 94, 61, 30}
(sim,lamb_da,p_rew,start,dose_quantity) =  (157, 0.9800000000000001, 9.8, 1, 0.5)
{14}
(sim,lamb_da,p_rew,start,dose_quantity) =  (157, 0.9800000000000001, 9.8, 1, 2)
{80}
(sim,lamb_da,p_rew,start,dose_quantity) =  (157, 0.9800000000000001, 9.8, 10, 0.5)
{0, 1, 66, 71, 82, 30, 87, 60, 61, 94}
(sim,lamb_da,p_rew,start,dose_quantity) =  (157, 0.9800000000000001, 9.8, 10, 2)
{34, 37, 74, 11, 46, 14, 50, 21, 8

(sim,lamb_da,p_rew,start,dose_quantity) =  (162, 0.9800000000000001, 9.8, 1, 2)
{3}
(sim,lamb_da,p_rew,start,dose_quantity) =  (162, 0.9800000000000001, 9.8, 10, 0.5)
{1, 39, 72, 10, 50, 82, 19, 29, 86, 93}
(sim,lamb_da,p_rew,start,dose_quantity) =  (162, 0.9800000000000001, 9.8, 10, 2)
{2, 7, 41, 73, 44, 46, 14, 52, 85, 95}
(sim,lamb_da,p_rew,start,dose_quantity) =  (162, 9.8, 98.0, 1, 0.5)
{16}
(sim,lamb_da,p_rew,start,dose_quantity) =  (162, 9.8, 98.0, 1, 2)
{33}
(sim,lamb_da,p_rew,start,dose_quantity) =  (162, 9.8, 98.0, 10, 0.5)
{39, 72, 42, 43, 15, 29, 93, 87, 57, 61}
(sim,lamb_da,p_rew,start,dose_quantity) =  (162, 9.8, 98.0, 10, 2)
{0, 2, 68, 73, 41, 44, 14, 46, 52, 95}
(sim,lamb_da,p_rew,start,dose_quantity) =  (163, 0.0098, 0.098, 1, 0.5)
{71}
(sim,lamb_da,p_rew,start,dose_quantity) =  (163, 0.0098, 0.098, 1, 2)
{87}
(sim,lamb_da,p_rew,start,dose_quantity) =  (163, 0.0098, 0.098, 10, 0.5)
{1, 68, 7, 73, 12, 47, 82, 88, 94, 62}
(sim,lamb_da,p_rew,start,dose_quantity) =  (163, 

(sim,lamb_da,p_rew,start,dose_quantity) =  (168, 0.0098, 0.098, 1, 0.5)
{38}
(sim,lamb_da,p_rew,start,dose_quantity) =  (168, 0.0098, 0.098, 1, 2)
{9}
(sim,lamb_da,p_rew,start,dose_quantity) =  (168, 0.0098, 0.098, 10, 0.5)
{64, 32, 34, 97, 68, 11, 78, 93, 26, 29}
(sim,lamb_da,p_rew,start,dose_quantity) =  (168, 0.0098, 0.098, 10, 2)
{32, 64, 0, 97, 11, 43, 78, 93, 26, 29}
(sim,lamb_da,p_rew,start,dose_quantity) =  (168, 0.098, 0.98, 1, 0.5)
{62}
(sim,lamb_da,p_rew,start,dose_quantity) =  (168, 0.098, 0.98, 1, 2)
{16}
(sim,lamb_da,p_rew,start,dose_quantity) =  (168, 0.098, 0.98, 10, 0.5)
{66, 99, 37, 38, 46, 15, 18, 53, 54, 89}
(sim,lamb_da,p_rew,start,dose_quantity) =  (168, 0.098, 0.98, 10, 2)
{4, 69, 7, 73, 43, 13, 16, 84, 92, 63}
(sim,lamb_da,p_rew,start,dose_quantity) =  (168, 0.9800000000000001, 9.8, 1, 0.5)
{53}
(sim,lamb_da,p_rew,start,dose_quantity) =  (168, 0.9800000000000001, 9.8, 1, 2)
{95}
(sim,lamb_da,p_rew,start,dose_quantity) =  (168, 0.9800000000000001, 9.8, 10, 0.5)
{